# 在 Amazon Bedrock AgentCore Runtime 中托管使用 Amazon Bedrock 模型的 LangGraph 代理

## 概述

在本教程中，我们将学习如何使用 Amazon Bedrock AgentCore Runtime 托管您现有的代理。

我们将重点关注使用 Amazon Bedrock 模型的 LangGraph 示例。有关使用 Amazon Bedrock 模型的 Strands Agents 示例，请查看[这里](../01-strands-with-bedrock-model)，
有关使用 OpenAI 模型的 Strands Agents 示例，请查看[这里](../03-strands-with-openai-model)。

### 教程详情

| 信息             | 详情                                                                      |
|:--------------------|:-----------------------------------------------------------------------------|
| 教程类型       | 对话式                                                               |
| 代理类型          | 单一                                                                       |
| 代理框架   | LangGraph                                                                    |
| LLM 模型           | Anthropic Claude Sonnet 3                                                    |
| 教程组件 | 在 AgentCore Runtime 上托管代理。使用 LangGraph 和 Amazon Bedrock 模型 |
| 教程领域   | 跨领域                                                               |
| 示例复杂度  | 简单                                                                         |
| 使用的 SDK            | Amazon BedrockAgentCore Python SDK 和 boto3                                 |

### 教程架构

在本教程中，我们将描述如何将现有代理部署到 AgentCore runtime。

出于演示目的，我们将使用带有 Amazon Bedrock 模型的 LangGraph 代理

在我们的示例中，我们将使用一个非常简单的代理，它有两个工具：`get_weather` 和 `get_time`。

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="50%"/>
</div>

### 教程主要特点

* 在 Amazon Bedrock AgentCore Runtime 上托管代理
* 使用 Amazon Bedrock 模型
* 使用 LangGraph


## 先决条件

要执行本教程，您需要：
* Python 3.10+
* AWS 凭证
* Amazon Bedrock AgentCore SDK
* LangGraph
* Docker 运行中

In [ ]:
#!uv add -r requirements.txt --active

## 创建代理并在本地进行实验

在将代理部署到 AgentCore Runtime 之前，让我们先在本地开发和运行它们进行实验。

对于生产级代理应用程序，我们需要将代理创建过程与代理调用过程分离。使用 AgentCore Runtime，我们将使用 `@app.entrypoint` 装饰器装饰代理的调用部分，并将其作为我们运行时的入口点。让我们首先看看在实验阶段如何开发每个代理。

这里的架构将如下所示：

<div style="text-align:left">
    <img src="images/architecture_local.png" width="50%"/>
</div>

In [ ]:
%%writefile langgraph_bedrock.py
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
import argparse
import json
import operator
import math

# Create calculator tool
@tool
def calculator(expression: str) -> str:
    """
    Calculate the result of a mathematical expression.
    
    Args:
        expression: A mathematical expression as a string (e.g., "2 + 3 * 4", "sqrt(16)", "sin(pi/2)")
    
    Returns:
        The result of the calculation as a string
    """
    try:
        # Define safe functions that can be used in expressions
        safe_dict = {
            "__builtins__": {},
            "abs": abs, "round": round, "min": min, "max": max,
            "sum": sum, "pow": pow,
            # Math functions
            "sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan,
            "log": math.log, "log10": math.log10, "exp": math.exp,
            "pi": math.pi, "e": math.e,
            "ceil": math.ceil, "floor": math.floor,
            "degrees": math.degrees, "radians": math.radians,
            # Basic operators (for explicit use)
            "add": operator.add, "sub": operator.sub,
            "mul": operator.mul, "truediv": operator.truediv,
        }
        
        # Evaluate the expression safely
        result = eval(expression, safe_dict)
        return str(result)
        
    except ZeroDivisionError:
        return "Error: Division by zero"
    except ValueError as e:
        return f"Error: Invalid value - {str(e)}"
    except SyntaxError:
        return "Error: Invalid mathematical expression"
    except Exception as e:
        return f"Error: {str(e)}"

# Create a custom weather tool
@tool
def weather():
    """Get weather"""  # Dummy implementation
    return "sunny"

# Define the agent using manual LangGraph construction
def create_agent():
    """Create and configure the LangGraph agent"""
    from langchain_aws import ChatBedrock
    
    # Initialize your LLM (adjust model and parameters as needed)
    llm = ChatBedrock(
        model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # or your preferred model
        model_kwargs={"temperature": 0.1}
    )
    
    # Bind tools to the LLM
    tools = [calculator, weather]
    llm_with_tools = llm.bind_tools(tools)
    
    # System message
    system_message = "You're a helpful assistant. You can do simple math calculation, and tell the weather."
    
    # Define the chatbot node
    def chatbot(state: MessagesState):
        # Add system message if not already present
        messages = state["messages"]
        if not messages or not isinstance(messages[0], SystemMessage):
            messages = [SystemMessage(content=system_message)] + messages
        
        response = llm_with_tools.invoke(messages)
        return {"messages": [response]}
    
    # Create the graph
    graph_builder = StateGraph(MessagesState)
    
    # Add nodes
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("tools", ToolNode(tools))
    
    # Add edges
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    
    # Set entry point
    graph_builder.set_entry_point("chatbot")
    
    # Compile the graph
    return graph_builder.compile()

# Initialize the agent
agent = create_agent()

def langgraph_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    
    # Create the input in the format expected by LangGraph
    response = agent.invoke({"messages": [HumanMessage(content=user_input)]})
    
    # Extract the final message content
    return response["messages"][-1].content

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = langgraph_bedrock(json.loads(args.payload))
    print(response)

#### 调用本地代理

In [ ]:
!python langgraph_bedrock.py '{"prompt": "What is the weather now?"}'

## 准备代理以部署到 AgentCore Runtime

现在让我们将代理部署到 AgentCore Runtime。为此，我们需要：
* 使用 `from bedrock_agentcore.runtime import BedrockAgentCoreApp` 导入 Runtime App
* 在代码中使用 `app = BedrockAgentCoreApp()` 初始化 App
* 使用 `@app.entrypoint` 装饰器装饰调用函数
* 让 AgentCoreRuntime 使用 `app.run()` 控制代理的运行

### 使用 Amazon Bedrock 模型的 LangGraph
让我们从使用 Amazon Bedrock 模型的 LangGraph 开始。其他使用不同框架和模型的示例可在父目录中找到

In [ ]:
%%writefile langgraph_bedrock.py
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from bedrock_agentcore.runtime import BedrockAgentCoreApp
import argparse
import json
import operator
import math

app = BedrockAgentCoreApp()

# Create calculator tool
@tool
def calculator(expression: str) -> str:
    """
    Calculate the result of a mathematical expression.
    
    Args:
        expression: A mathematical expression as a string (e.g., "2 + 3 * 4", "sqrt(16)", "sin(pi/2)")
    
    Returns:
        The result of the calculation as a string
    """
    try:
        # Define safe functions that can be used in expressions
        safe_dict = {
            "__builtins__": {},
            "abs": abs, "round": round, "min": min, "max": max,
            "sum": sum, "pow": pow,
            # Math functions
            "sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan,
            "log": math.log, "log10": math.log10, "exp": math.exp,
            "pi": math.pi, "e": math.e,
            "ceil": math.ceil, "floor": math.floor,
            "degrees": math.degrees, "radians": math.radians,
            # Basic operators (for explicit use)
            "add": operator.add, "sub": operator.sub,
            "mul": operator.mul, "truediv": operator.truediv,
        }
        
        # Evaluate the expression safely
        result = eval(expression, safe_dict)
        return str(result)
        
    except ZeroDivisionError:
        return "Error: Division by zero"
    except ValueError as e:
        return f"Error: Invalid value - {str(e)}"
    except SyntaxError:
        return "Error: Invalid mathematical expression"
    except Exception as e:
        return f"Error: {str(e)}"

# Create a custom weather tool
@tool
def weather():
    """Get weather"""  # Dummy implementation
    return "sunny"

# Define the agent using manual LangGraph construction
def create_agent():
    """Create and configure the LangGraph agent"""
    from langchain_aws import ChatBedrock
    
    # Initialize your LLM (adjust model and parameters as needed)
    llm = ChatBedrock(
        model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # or your preferred model
        model_kwargs={"temperature": 0.1}
    )
    
    # Bind tools to the LLM
    tools = [calculator, weather]
    llm_with_tools = llm.bind_tools(tools)
    
    # System message
    system_message = "You're a helpful assistant. You can do simple math calculation, and tell the weather."
    
    # Define the chatbot node
    def chatbot(state: MessagesState):
        # Add system message if not already present
        messages = state["messages"]
        if not messages or not isinstance(messages[0], SystemMessage):
            messages = [SystemMessage(content=system_message)] + messages
        
        response = llm_with_tools.invoke(messages)
        return {"messages": [response]}
    
    # Create the graph
    graph_builder = StateGraph(MessagesState)
    
    # Add nodes
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("tools", ToolNode(tools))
    
    # Add edges
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    
    # Set entry point
    graph_builder.set_entry_point("chatbot")
    
    # Compile the graph
    return graph_builder.compile()

# Initialize the agent
agent = create_agent()

@app.entrypoint
def langgraph_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    
    # Create the input in the format expected by LangGraph
    response = agent.invoke({"messages": [HumanMessage(content=user_input)]})
    
    # Extract the final message content
    return response["messages"][-1].content

if __name__ == "__main__":
    app.run()

## 幕后发生了什么？

当您使用 `BedrockAgentCoreApp` 时，它会自动：

* 创建一个监听 8080 端口的 HTTP 服务器
* 实现处理代理需求所需的 `/invocations` 端点
* 实现用于健康检查的 `/ping` 端点（对于异步代理非常重要）
* 处理适当的内容类型和响应格式
* 根据 AWS 标准管理错误处理

## 将代理部署到 AgentCore Runtime

`CreateAgentRuntime` 操作支持全面的配置选项，让您可以指定容器镜像、环境变量和加密设置。您还可以配置协议设置（HTTP、MCP）和授权机制，以控制客户端如何与代理通信。

**注意：** 操作最佳实践是将代码打包为容器并使用 CI/CD 管道和 IaC 推送到 ECR

在本教程中，我们将使用 Amazon Bedrock AgentCode Python SDK 轻松打包您的构件并将它们部署到 AgentCore runtime。

### 创建运行时角色

在开始之前，让我们为 AgentCore Runtime 创建一个 IAM 角色。我们将使用为您预先开发的 utils 函数来实现这一点。

In [ ]:
import sys
import os
import json
import boto3

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

# Navigate up to the utils.py location
utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)

from utils import create_agentcore_role

agent_name="langgraph_bedrock"
agentcore_iam_role = create_agentcore_role(agent_name=agent_name)

### 配置 AgentCore Runtime 部署

接下来，我们将使用我们的启动工具包配置 AgentCore Runtime 部署，包括入口点、我们刚刚创建的执行角色和一个需求文件。我们还将配置启动工具包在启动时自动创建 Amazon ECR 存储库。

在配置步骤中，将根据您的应用程序代码生成 docker 文件

<div style="text-align:left">
    <img src="images/configure.png" width="40%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="langgraph_bedrock.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region
)
response

### 将代理启动到 AgentCore Runtime

现在我们已经有了 docker 文件，让我们将代理启动到 AgentCore Runtime。这将创建 Amazon ECR 存储库和 AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()
launch_result

### 检查 AgentCore Runtime 状态
现在我们已经部署了 AgentCore Runtime，让我们检查它的部署状态

In [ ]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### 调用 AgentCore Runtime

最后，我们可以使用有效负载调用我们的 AgentCore Runtime

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "How much is 2+2?"})
invoke_response

### 处理调用结果

我们现在可以处理我们的调用结果，以将其包含在应用程序中

In [ ]:
from IPython.display import Markdown, display
response_text = json.loads(invoke_response['response'][0].decode("utf-8"))
display(Markdown(response_text))

### 使用 boto3 调用 AgentCore Runtime

现在您的 AgentCore Runtime 已创建，您可以使用任何 AWS SDK 调用它。例如，您可以使用 boto3 的 `invoke_agent_runtime` 方法。

In [ ]:
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What is the weather now?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                logger.info(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

## 清理（可选）

现在让我们清理创建的 AgentCore Runtime

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

iam_client = boto3.client('iam')

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

policies = iam_client.list_role_policies(
    RoleName=agentcore_iam_role['Role']['RoleName'],
    MaxItems=100
)

for policy_name in policies['PolicyNames']:
    iam_client.delete_role_policy(
        RoleName=agentcore_role_name,
        PolicyName=policy_name
    )
iam_response = iam_client.delete_role(
    RoleName=agentcore_role_name
)

# 恭喜！